In [ ]:
import numpy as np

In [ ]:
from torch import nn, Tensor
from dataclasses import dataclass


# hormone = 20 [id], 
# neuron = [activation_threshold, activation_decay, signal_multiplier, signal_delay, connection_affinity, spontaneity, ambient_hormone_release_rate, activation_hormone_release, hormone_decay_rate, hormone_range]
#          [offsets...] x 8 max simultaneous hormone effects
#          [decay_rates...] x 8
#          (d x 10) NL (10 x 20) - offsets
#          

# neuron_division_matrix = (d x d)

@dataclass
class Neuron:
  # FUNCTIONAL PARAMETERS
  # - activation parameters
  activation_threshold: float
  activation_cooldown: float
  signal_strength: float
  mitosis_qualification: float  # when this reaches 1.0, mitosis occurs
  # - other states
  
  # INTERNAL STATE
  latent_state: Tensor  # produced exclusively by transforms defined in the genome
  total_receptivity: float  # summed connectivity coefficients of incoming connections
  total_emissivity: float  # summed connectivity coefficients of outgoing connections
  
  # HIDDEN STATE (for implementation purposes only)
  position: Tensor  # 3d global position
  inputs: Tensor  # (MAX_CONNECTIONS x 2), each connection includes a neuron ID and a connectivity coefficient
  outputs: Tensor  # same structure as inputs
  
@dataclass
class Genome:
  pluripotent_latent_state: Tensor  # initial latent state for the first neuron(s)
  latent_to_parameters: nn.Module  # takes in internal state, outputs activation parameters
  passive_transform: nn.Module  # applied continuously
  activation_transform: nn.Module  # takes in internal state, outputs new latent state
  neurotransmitter_transform: nn.Module  # takes in internal state and hormone, outputs new latent state
  connectivity_coefficient: nn.Module  # takes in internal state pair and relative position (emitter and receiver and emitter to receiver), outputs receptivity of latter [0,1]
  mitosis_results: nn.Module  # takes in internal state, outputs new latent state and daughter latent state and mitosis direction
  
  # MUTABILITY
  # ... (modules of the same dimensions as genome which define the rate of genetic drift)
  